In [ ]:
!pip install transformers datasets torch nltk
!pip install pdfminer.six
!pip install python-docx

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import re
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from pdfminer.high_level import extract_text as pdf_extract_text
from docx import Document

In [4]:
# Function to extract text from files
def extract_text_from_file(file_path):
    if file_path.endswith('.pdf'):
        return pdf_extract_text(file_path)
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    return ""

# Load all text files from specified directory
def load_data_from_directory(directory):
    all_texts = []
    for file_path in glob.glob(os.path.join(directory, '*')):
        text = extract_text_from_file(file_path)
        all_texts.append(text)
    return all_texts

In [5]:
import re

# Function to clean and preprocess text
def clean_text(text):

    # Remove timestamps in the format (7:41PM, 7/17/2016)
    text = re.sub(r'\(\d{1,2}:\d{2}(?:AM|PM), \d{1,2}/\d{1,2}/\d{4}\)', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    # Trim leading/trailing whitespace
    text = text.strip()
    return text

In [6]:
# Load and clean data
data_directory = '/content/drive/MyDrive/finetuning data/'
texts = load_data_from_directory(data_directory)
cleaned_texts = [clean_text(text) for text in texts if text]

In [ ]:
# download cleaned text in .txt file
with open('/content/cleaned_texts.txt', 'w') as f:
  for text in cleaned_texts:
    f.write(text + '\n')

from google.colab import files
files.download('cleaned_texts.txt')